In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
#Read main data
data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\data_values.xlsx',sheet_name='data_transpose')
rainfall_data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\rainfall_data.xlsx',sheet_name='Sheet1')

# Read data necessary for normalization
ind_type=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\indicator_type.xlsx',sheet_name='Sheet1')

In [3]:
#defining normalization function for (dam capicity)/(surface rwr)
def damcap_normalize(ser1,ser2):
    result=ser1*0
    length=ser1.size
    for i in range(length):
        if ser1[i]>=ser2[i]:
            result[i]=(ser1.max()-ser1[i])/(ser1.max()-ser2[i])
        else:
            result[i]=(ser1[i]-ser1.min())/(ser2[i]-ser1.min())
        if result[i]<0.01:
            result[i]=0.01
    return result  

#defining aggregation functions
def aggregate(df,a=0.5):
    size=len(df.columns)
    result=(df.product(axis=1)**(1/size))*a+df.mean(axis=1)*(1-a)
    return result

#defining weighted aggregation functions
def weighted_addminagg(dfi,dfw,a=0.5):
    weight_array=np.array(dfw)[0]
    dfmin=dfi.min(axis=1)
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    result=a*dfmin+(1-a)*weighted_sum
    return result

def weighted_sum(dfi,dfw):
    weight_array=np.array(dfw)[0]
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    return weighted_sum

def geomean(dfi):
    size=len(dfi.columns)
    weighted_geomean=dfi.pow(weight_array,axis='columns').product(axis='columns')
    return weighted_geomean

def weighted_geomean(dfi,dfw):
    weight_array=np.array(dfw)[0]
    weighted_geomean=dfi.pow(weight_array,axis='columns').product(axis='columns')
    return weighted_geomean

def weighted_addgeo(dfi,dfw,a=0.5):
    weight_array=np.array(dfw)[0]
    weighted_geomean=dfi.pow(weight_array,axis='columns').product(axis='columns')
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    result= a*weighted_geomean+(1-a)*weighted_sum
    return result

In [4]:
#selecting rainfall data after 1366 (nearly 30 years)
rainfall_modified=rainfall_data.loc[(rainfall_data['syear']>1366) & (rainfall_data['syear']<1399)]

#making a dictionary of provinces and their respective data
pr=list(rainfall_modified.ostan.unique())
prdict={elem:pd.DataFrame() for elem in pr}
for key in prdict.keys():
    prdict[key]=rainfall_modified[:][rainfall_modified.ostan==key]
    
#making a dictionary consisting of province names as keys and annual precipitation as respective values
annualrain_dict={x:pd.DataFrame() for x in pr}
for key in annualrain_dict.keys():
    annualrain_dict[key]=prdict[key].groupby('syear').precnew.sum()

#calculating coefficients of variation
interannual_varicoef={x:pd.DataFrame() for x in pr}
for key in interannual_varicoef.keys():
    interannual_varicoef[key]=annualrain_dict[key].std()/annualrain_dict[key].mean()
monthly_varicoef={x:pd.DataFrame() for x in pr}
for key in monthly_varicoef.keys():
    monthly_varicoef[key]=prdict[key].precnew.std()/prdict[key].precnew.mean()
annualevap_varicoef={x:pd.DataFrame() for x in pr}

#converting monthly coefficient of variation to dataframe and sorting it based on original data
monthlyvaricoeff_df = pd.DataFrame(monthly_varicoef.items(),columns=['province', 'monthly_varicoeff'])
monthlyvaricoeff_df = monthlyvaricoeff_df.drop([31])
monthlyvaricoeff_df = monthlyvaricoeff_df.set_index('province')
monthlyvaricoeff_df = monthlyvaricoeff_df.reindex(index=data['province'])
monthlyvaricoeff_df = monthlyvaricoeff_df.reset_index()

#converting annual coefficient of variation to dataframe and sorting it based on original data
intanvaricoeff_df = pd.DataFrame(interannual_varicoef.items(),columns=['province', 'rain_coeff_variability'])
intanvaricoeff_df = intanvaricoeff_df.drop([31])
intanvaricoeff_df = intanvaricoeff_df.set_index('province')
intanvaricoeff_df = intanvaricoeff_df.reindex(index=data['province'])
intanvaricoeff_df = intanvaricoeff_df.reset_index()

# adding calculated coefficients of variation to the data
data['rain_coeff_variation']=intanvaricoeff_df['rain_coeff_variability']
data['monthly_varicoeff']= monthlyvaricoeff_df['monthly_varicoeff']

In [41]:
#Creating a dataframe consisting of necessary variables for the calculation of sub-indicators
variables=pd.DataFrame()
variables['province']=data.province
variables['irwr']=data.precipitation-data.evaporation
variables['surf_irwr']=(data.precipitation-data.evaporation)*data.runoff_coeff
variables['withdraw_surf']=data.iloc[:,5:8].sum(axis=1)
variables['gw_irwr']=(data.precipitation-data.evaporation)*(1-data.runoff_coeff)
variables['withdraw_gw']=data.iloc[:,8:11].sum(axis=1)
variables['withdraw_agr']=data.withdraw_gw_agr+data.withdraw_surf_agr
variables['withdraw_ind']=data.withdraw_gw_ind+data.withdraw_surf_ind
variables['access_sanitation_total']=(data.access_sanitation_urban*data.urban_pop_ratio+data.access_sanitation_rural*(1-data.urban_pop_ratio))
variables['deficit_gw_annual']=data.annual_gw_variation*(-1)
variables['deficit_gw_aggregate']=data.aggregate_gw_variation*(-1)
variables['withdraw_total']=np.NaN
variables['withdraw_total']=data[['withdraw_surf_agr','withdraw_surf_ind','withdraw_surf_dom','withdraw_gw_agr','withdraw_gw_ind','withdraw_gw_dom']].sum(axis=1)
variables['agr_withdraw_ratio']=variables['withdraw_agr']/variables['withdraw_total']
variables['access_sanitation_total']=data['access_sanitation_urban']*data['urban_pop_ratio']+data['access_sanitation_rural']*(1-data['urban_pop_ratio'])

#setting negative deficit values equal to zero
variables.loc[variables['deficit_gw_annual'] < 0 ,'deficit_gw_annual']=0
variables.loc[variables['deficit_gw_aggregate'] < 0 ,'deficit_gw_aggregate']=0
variables['withdraw_gw_allowable']=variables.gw_irwr-(variables.deficit_gw_aggregate/17)
variables['withdraw_agr_decrease']=variables.withdraw_gw-variables.withdraw_gw_allowable
variables.loc[variables['withdraw_agr_decrease']<0,'withdraw_agr_decrease']=0
variables['agrwat_lost_ratio']=variables.withdraw_agr_decrease/variables.withdraw_agr
variables[['province','irwr']]

,province,irwr
0,azsh,3366.7100
1,azgh,4582.9419
2,arda,1173.7600
3,esfe,5203.0813
4,albo,1138.4900
5,ilam,2060.3665
6,bush,1023.8482
7,tehr,1340.7200
8,chba,7540.2905
9,khoj,1293.9195


In [6]:
# calculating trade-off ratio between modern irrigation and employment lost

water_saved_perhec=variables['withdraw_agr']/(2*data['land_irrig_tot']-data['land_irrig_modern'])

# wwdpmii= water withdrawal decrease per modern irrigation increase (percent)
wwdpmii=water_saved_perhec*0.01*data['land_irrig_tot']
#wdn=water decrease necessary for compensating the effect of lowering 1 percent modern irrigation
wdn=variables['withdraw_agr_decrease']+wwdpmii
#elsftr=employment lost for tradeoff ratio & mitr=modern irrigation for tradeoff ratio
eltr=wdn*data.agr_employment/variables.withdraw_agr
mitr=data.land_irrig_modern/data.land_irrig_tot-0.01
eln=variables['withdraw_agr_decrease']*data.agr_employment/variables.withdraw_agr
tradeoff_ratio=(eltr-eln)*100/18


In [39]:
#Creating a dataframe for the sub-indicators
#Dimension 1: Resources
sub_indicators=pd.DataFrame()
sub_indicators['rain_coeff_variation']=data['rain_coeff_variation']
sub_indicators['monthly_varicoeff']=data['monthly_varicoeff']
sub_indicators['gw_agdef_gwrwr']=variables.deficit_gw_aggregate/variables.gw_irwr
sub_indicators['gw_andef_withdraw']=variables.deficit_gw_annual/variables.withdraw_gw
sub_indicators['agr_dependency_gw']=data.withdraw_gw_agr/variables.withdraw_agr
sub_indicators['ind_dependency_gw']=data.withdraw_gw_ind/variables.withdraw_ind
sub_indicators['urbanwat_dependency_gw']=data.withdraw_urban_gw/data.produced_urban_wat
sub_indicators['ruralwat_dependency_gw']=data.withdraw_rural_gw/data.produced_rural_wat
sub_indicators['withdraw_surf_ratio']=variables.withdraw_surf/variables.surf_irwr
sub_indicators['withdraw_gw_ratio']=variables.withdraw_gw/variables.gw_irwr
sub_indicators['anomaly_rain']=abs(data['anomaly_rain'])
sub_indicators['anomaly_temp']=data['anomaly_temp']
sub_indicators['irwr_percap']=(variables.irwr*(10**6)/data.population)/(1+data['pop_growth']/100)
#Dimension 2: access

sub_indicators['access_wat_urban']=data['access_wat_urban']
sub_indicators['access_wat_rural']=data['access_wat_rural']
sub_indicators['under_stress_pop']=data['under_stress_pop']
sub_indicators['access_sanitation_urban']=data['access_sanitation_urban']
sub_indicators['access_sanitation_rural']=data['access_sanitation_rural']
sub_indicators['treated_municipal_wastewater']=data.waste_facility_cap/data.daily_produced_waste
sub_indicators['quality_proxy']=data.urban_fam_treatwat/data.total_urban_fam
sub_indicators['damcap_rwr_ratio']=data.dam_cap/variables.surf_irwr

#Dimension 3:Economy
sub_indicators['efficiency_agr']=data.agr_added_value/variables.withdraw_agr
sub_indicators['modern_irrig']=data.land_irrig_modern/data.land_irrig_tot
sub_indicators['employment_lost_agr']=variables.agrwat_lost_ratio*data.agr_employment
sub_indicators['unaccounted_wat_urban']=data['unaccounted_wat_urban']
sub_indicators['unaccounted_wat_rural']=data['unaccounted_wat_rural']
sub_indicators['efficiency_ind']=data.ind_added_value/variables.withdraw_ind

sub_indicators.index=variables['province']
sub_indicators.loc['khuz','damcap_rwr_ratio']=1
sub_indicators.loc['tehr',:]

rain_coeff_variation              0.210653
monthly_varicoeff                 0.965221
gw_agdef_gwrwr                    7.707304
gw_andef_withdraw                 0.064889
agr_dependency_gw                 0.600733
ind_dependency_gw                 0.879666
urbanwat_dependency_gw            0.533063
ruralwat_dependency_gw            1.000000
withdraw_surf_ratio               2.332092
withdraw_gw_ratio                 4.544267
anomaly_rain                     13.088950
anomaly_temp                      0.507672
irwr_percap                      92.793813
access_wat_urban                 99.900000
access_wat_rural                 78.000000
under_stress_pop                  0.183542
access_sanitation_urban          46.580000
access_sanitation_rural           2.400000
treated_municipal_wastewater      0.763453
quality_proxy                     0.231550
damcap_rwr_ratio                  1.678330
efficiency_agr                    0.663583
modern_irrig                      0.300769
employment_

In [8]:
#Normalizing Data
normalized=sub_indicators*0
max_allowable_withdraw=variables['withdraw_gw_allowable']/variables['gw_irwr']
max_allowable_withdraw.index=sub_indicators.index

for col in ind_type.columns:
    i=ind_type.columns.get_loc(col)
    if ind_type.loc[2,col]=='b': # The bigger the better indicators
        normalized.loc[sub_indicators[col]>=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]<=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]>ind_type.loc[1,col])& (sub_indicators[col]<ind_type.loc[0,col])
        normalized.loc[cond,col]=(sub_indicators.loc[cond,col]-ind_type.loc[1,col])/(ind_type.loc[0,col]-ind_type.loc[1,col])
    elif ind_type.loc[2,col]=='l':      # The lower the better indicators
        normalized.loc[sub_indicators[col]<=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]>=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]<ind_type.loc[1,col])& (sub_indicators[col]>ind_type.loc[0,col])
        normalized.loc[cond,col]=(ind_type.loc[1,col]-sub_indicators.loc[cond,col])/(ind_type.loc[1,col]-ind_type.loc[0,col])
    elif ind_type.loc[2,col]=='diff':    # GW withdrawal to rwr ratio normalization
        cond1=(sub_indicators[col] > max_allowable_withdraw)
        normalized.loc[cond1,col]=0.01
        cond2=sub_indicators[col]<0.25
        normalized.loc[cond2,col]=1
        cond3=(sub_indicators[col] < max_allowable_withdraw) & (sub_indicators[col]>0.25)
        normalized.loc[cond3,col]=(max_allowable_withdraw.loc[cond3]-sub_indicators.loc[cond3,col])/(max_allowable_withdraw.loc[cond3]-0.25)
        

In [9]:
# dam capacity to surface rwr ratio normalization    
variation_coeff_agg=pd.Series(normalized[['rain_coeff_variation','monthly_varicoeff']].mean(axis=1),index=variables['province'])
variation_coeff_bins=pd.cut(variation_coeff_agg,4,labels=['Q1','Q2','Q3','Q4'])
ideal_damcap=pd.Series(index=variables['province'],dtype='float64') 
ideal_damcap[variation_coeff_bins=='Q1']=1
ideal_damcap[variation_coeff_bins=='Q2']=0.9
ideal_damcap[variation_coeff_bins=='Q3']=0.8
ideal_damcap[variation_coeff_bins=='Q4']=0.7
normalized['damcap_rwr_ratio']=damcap_normalize(sub_indicators['damcap_rwr_ratio'],ideal_damcap)   
normalized.loc['khuz','damcap_rwr_ratio']=1

# modifying modern irrigation indicator vlues based on aggregate GW reservoir deficits
gw_deficit_agg=pd.Series(aggregate(normalized[['gw_agdef_gwrwr','gw_andef_withdraw']]),index=variables['province'])
gw_deficit_bins=pd.qcut(gw_deficit_agg,4,labels=['Q1','Q2','Q3','Q4'])
gw_modifier=pd.Series(index=variables['province'],dtype='float64')
gw_modifier[gw_deficit_bins=='Q1']=0.8
gw_modifier[gw_deficit_bins=='Q2']=0.9
gw_modifier[gw_deficit_bins=='Q3']=1
gw_modifier[gw_deficit_bins=='Q4']=1
normalized['modern_irrig']=normalized['modern_irrig'].multiply(gw_modifier)



In [10]:
# reading AHP weights from csv files
eco_weights=pd.read_csv('eco_modified_weights.csv')
acc_weights=pd.read_csv('acc_modified_weights.csv')
res_weights=pd.read_csv('res_modified_weights.csv')
wsi_weights=pd.read_csv('wsi_modified_weights.csv')

eco_weights.drop(columns='Unnamed: 0',inplace=True)
arrayeco=[['E1','E1','E1','E2','E2','E3','E3','E3'],list(eco_weights.iloc[0,:])]
eco_weights.columns=pd.MultiIndex.from_arrays(arrayeco, names=('Aggregation Level', 'Indicator'))
eco_weights.drop(index=0,inplace=True)
eco_weights.index=['Weight']
eco_weights=eco_weights.astype('float')

acc_weights.drop(columns='Unnamed: 0',inplace=True)
arrayacc=[['A1','A1','A1','A2','A2','A2','A3','A3','A3','A3'],list(acc_weights.iloc[0,:])]
acc_weights.columns=pd.MultiIndex.from_arrays(arrayacc, names=('Aggregation Level', 'Indicator'))
acc_weights.drop(index=0,inplace=True)
acc_weights.index=['Weight']
acc_weights=acc_weights.astype('float')

res_weights.drop(columns='Unnamed: 0',inplace=True)
arrayres=[['R1','R1','R2','R2','R3','R3','R3','R3','R4','R4','R5','R5','R5','R6','R6','R7','R7','R7','R7'],list(res_weights.iloc[0,:])]
res_weights.columns=pd.MultiIndex.from_arrays(arrayres, names=('Aggregation Level', 'Indicator'))
res_weights.drop(index=0,inplace=True)
res_weights.index=['Weight']
res_weights=res_weights.astype('float')

wsi_weights.drop(columns='Unnamed: 0',inplace=True)
arraywsi=[['WSI','WSI','WSI'],list(wsi_weights.iloc[0,:])]
wsi_weights.columns=pd.MultiIndex.from_arrays(arraywsi, names=('Aggregation Level', 'Indicator'))
wsi_weights.drop(index=0,inplace=True)
wsi_weights.index=['Weight']
wsi_weights=wsi_weights.astype('float')
ahp_weights=wsi_weights.join(res_weights.join(acc_weights.join(eco_weights,how='outer'),how='outer'),how='outer')

In [30]:
res_sub_columns=['ACV','MCV','Dag','Dan','GWDagr','GWDind','GWDurb','GWDrur','SWS','GWS','APA','ATA','IRWR']
acc_sub_columns=['WAurb','WArur','USP','SAurb','SArur','TPC','WQ','DC']
eco_sub_columns=['AE','MI','EL','NRWurb','NRWrur','IE']
sub_columns1_list= ['R1','R1','R2','R2','R3','R3','R3','R3','R4','R4','R5','R5','R7']+['A1','A1','A1','A2','A2','A2','A3','A3']+['E1','E1','E1','E2','E2','E3']
sub_columns2_list=res_sub_columns+acc_sub_columns+eco_sub_columns
arraycolumns=[sub_columns1_list]+[sub_columns2_list]
normalized_ahp=pd.DataFrame(normalized).round(decimals=4)
normalized_ahp.columns=pd.MultiIndex.from_arrays(arraycolumns, names=('Aggregation Level', 'Indicator'))
normalized_ahp.loc[['khuz','kohbu'],'R2']

Indicator,Dag,Dan
province,,
khuz,0.6817,0.0100
kohbu,0.9919,0.6417


In [ ]:
normalized_lvl2=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
normalized_lvl2.columns=pd.MultiIndex.from_arrays([['R5'],['CV']], names=('Aggregation Level', 'Indicator'))
normalized_lvl2[('R5','CV')]=weighted_addminagg(normalized_ahp.loc[:,'R1'],ahp_weights['R1']).round(decimals=4)
normalized_lvl2[('R6','GD')]=weighted_addminagg(normalized_ahp.loc[:,'R2'],ahp_weights['R2']).round(decimals=4)
normalized_lvl2[('R6','GWD')]=weighted_addminagg(normalized_ahp.loc[:,'R3'],ahp_weights['R3']).round(decimals=4)

## Assessing water security using a mix of weighted_sum and minimum aggregation rules

#  calculating indicators 
indicators_weighted=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
indicators_weighted.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_weighted[('Resource','IRWR')]=normalized_ahp.loc[:,('R7','IRWR')]
indicators_weighted[('Resource','CC')]=weighted_addminagg(normalized_lvl2['R5'].merge(normalized_ahp['R5'], left_index=True, right_index=True,how='outer'),ahp_weights['R5']).round(decimals=4)
indicators_weighted[('Resource','WS')]=weighted_addminagg(normalized_ahp.loc[:,'R4'],ahp_weights['R4']).round(decimals=4)
# if a province has low groundwater reservoir deficit then high dependency on groundwater is considered okay i.e. if GD> 0.8 then GWD not involved in aggregation
indicators_weighted.loc[normalized_lvl2[('R6','GD')]<0.8,('Resource','GW')]=weighted_addminagg(normalized_lvl2.loc[:,'R6'],ahp_weights['R6']).round(decimals=4)
indicators_weighted.loc[normalized_lvl2[('R6','GD')]>0.8,('Resource','GW')]=normalized_lvl2.loc[:,('R6','GD')]
indicators_weighted[('Access','DW')]=weighted_addminagg(normalized_ahp.loc[:,'A1'],ahp_weights['A1']).round(decimals=4)
indicators_weighted[('Access','WQ')]=normalized_ahp.loc[:,('A3','WQ')]
indicators_weighted[('Access','DC')]=normalized_ahp.loc[:,('A3','DC')]
indicators_weighted[('Access','SS')]=weighted_addminagg(normalized_ahp.loc[:,'A2'],ahp_weights['A2']).round(decimals=4)
indicators_weighted[('Economy','Eagr')]=weighted_addminagg(normalized_ahp.loc[:,'E1'],ahp_weights['E1']).round(decimals=4)
indicators_weighted[('Economy','IE')]=normalized_ahp.loc[:,('E3','IE')]
indicators_weighted[('Economy','NRW')]=weighted_addminagg(normalized_ahp.loc[:,'E2'],ahp_weights['E2']).round(decimals=4)

#  calculating dimensions & WSI
Dimensions_addminagg=pd.DataFrame(index=sub_indicators.index)
Dimensions_addminagg['Resource']=weighted_addminagg(indicators_weighted[('Resource')],ahp_weights['R7']).round(decimals=4)
Dimensions_addminagg['Access']=weighted_addminagg(indicators_weighted[('Access')],ahp_weights['A3']).round(decimals=4)
Dimensions_addminagg['Economy']=weighted_addminagg(indicators_weighted[('Economy')],ahp_weights['E3']).round(decimals=4)
Dimensions_addminagg['WSI']=weighted_addminagg(Dimensions_addminagg,ahp_weights['WSI']).round(decimals=4)
# Dimensions_addminagg.sort_values('WSI',ascending=False)

In [55]:
# correlation analysis
temp=indicators_weighted['Resource'].join(Dimensions_addminagg['Resource'])
temp.corr(method='kendall')

,IRWR,CC,WS,GW,Resource
IRWR,1.000000,-0.102192,0.560616,0.409209,0.792558
CC,-0.102192,1.000000,-0.095344,0.034446,0.015054
WS,0.560616,-0.095344,1.000000,0.362264,0.628404
GW,0.409209,0.034446,0.362264,1.000000,0.576965
Resource,0.792558,0.015054,0.628404,0.576965,1.000000


In [53]:
writer = pd.ExcelWriter('Normalized Indicators.xlsx', engine='xlsxwriter')
indicators_weighted.to_excel(writer,sheet_name='Indicators')
normalized_lvl2.to_excel(writer,sheet_name='Level 2 Sub_Indicators')
writer.save()

In [14]:
## Assessing water security using weighted_sum aggregation 
normalized_lvl2=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
normalized_lvl2.columns=pd.MultiIndex.from_arrays([['R5'],['CV']], names=('Aggregation Level', 'Indicator'))
normalized_lvl2[('R5','CV')]=weighted_sum(normalized_ahp.loc[:,'R1'],ahp_weights['R1']).round(decimals=4)
normalized_lvl2[('R6','GD')]=weighted_sum(normalized_ahp.loc[:,'R2'],ahp_weights['R2']).round(decimals=4)
normalized_lvl2[('R6','GWD')]=weighted_sum(normalized_ahp.loc[:,'R3'],ahp_weights['R3']).round(decimals=4)

indicators_weighted=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
indicators_weighted.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_weighted[('Resource','IRWR')]=normalized_ahp.loc[:,('R7','IRWR')]
indicators_weighted[('Resource','CC')]=weighted_sum(normalized_lvl2['R5'].merge(normalized_ahp['R5'], left_index=True, right_index=True,how='outer'),ahp_weights['R5'])
indicators_weighted[('Resource','WS')]=weighted_sum(normalized_ahp.loc[:,'R4'],ahp_weights['R4'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]<0.8,('Resource','GW')]=weighted_sum(normalized_lvl2.loc[:,'R6'],ahp_weights['R6'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]>0.8,('Resource','GW')]=normalized_lvl2.loc[:,('R6','GD')]
indicators_weighted[('Access','DW')]=weighted_sum(normalized_ahp.loc[:,'A1'],ahp_weights['A1'])
indicators_weighted[('Access','WQ')]=normalized_ahp.loc[:,('A3','WQ')]
indicators_weighted[('Access','DC')]=normalized_ahp.loc[:,('A3','DC')]
indicators_weighted[('Access','SS')]=weighted_sum(normalized_ahp.loc[:,'A2'],ahp_weights['A2'])
indicators_weighted[('Economy','Eagr')]=weighted_sum(normalized_ahp.loc[:,'E1'],ahp_weights['E1'])
indicators_weighted[('Economy','IE')]=normalized_ahp.loc[:,('E3','IE')]
indicators_weighted[('Economy','NRW')]=weighted_sum(normalized_ahp.loc[:,'E2'],ahp_weights['E2'])

#  calculating dimensions & WSI
Dimensions_weightedsum=pd.DataFrame(index=sub_indicators.index)
Dimensions_weightedsum['Resource']=weighted_sum(indicators_weighted[('Resource')],ahp_weights['R7'])
Dimensions_weightedsum['Access']=weighted_sum(indicators_weighted[('Access')],ahp_weights['A3'])
Dimensions_weightedsum['Economy']=weighted_sum(indicators_weighted[('Economy')],ahp_weights['E3'])
Dimensions_weightedsum['WSI']=weighted_sum(Dimensions_weightedsum,ahp_weights['WSI']).round(decimals=4)
Dimensions_weightedsum.sort_values('WSI',ascending=False)

,Resource,Access,Economy,WSI
province,,,,
lores,0.916019,0.615757,0.588077,0.7601
mazan,0.828225,0.555551,0.719407,0.7263
gilan,0.938836,0.608302,0.369598,0.7234
kord,0.850139,0.630280,0.528967,0.7189
chba,0.863598,0.674422,0.397265,0.7108
kohbu,0.880330,0.572624,0.489675,0.7091
ilam,0.835903,0.662277,0.401982,0.6944
kersh,0.770479,0.510514,0.407824,0.6189
azgh,0.554183,0.675111,0.379276,0.5525


In [15]:
# writer = pd.ExcelWriter('WSI_weighted.xlsx', engine='xlsxwriter')
# WSI.to_excel(writer, sheet_name = 'تجمیع ترکیبی')
# WSI2.to_excel(writer, sheet_name = 'تجمیع حسابی')
# writer.save()

In [16]:
## Assessing water security using a weighted_geomean aggregation rule
normalized_lvl2=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
normalized_lvl2.columns=pd.MultiIndex.from_arrays([['R5'],['CV']], names=('Aggregation Level', 'Indicator'))
normalized_lvl2[('R5','CV')]=weighted_geomean(normalized_ahp.loc[:,'R1'],ahp_weights['R1']).round(decimals=4)
normalized_lvl2[('R6','GD')]=weighted_geomean(normalized_ahp.loc[:,'R2'],ahp_weights['R2']).round(decimals=4)
normalized_lvl2[('R6','GWD')]=weighted_geomean(normalized_ahp.loc[:,'R3'],ahp_weights['R3']).round(decimals=4)

indicators_weighted=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
indicators_weighted.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_weighted[('Resource','IRWR')]=normalized_ahp.loc[:,('R7','IRWR')]
indicators_weighted[('Resource','CC')]=weighted_geomean(normalized_lvl2['R5'].merge(normalized_ahp['R5'], left_index=True, right_index=True,how='outer'),ahp_weights['R5'])
indicators_weighted[('Resource','WS')]=weighted_geomean(normalized_ahp.loc[:,'R4'],ahp_weights['R4'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]<0.8,('Resource','GW')]=weighted_geomean(normalized_lvl2.loc[:,'R6'],ahp_weights['R6'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]>0.8,('Resource','GW')]=normalized_lvl2.loc[:,('R6','GD')]
indicators_weighted[('Access','DW')]=weighted_geomean(normalized_ahp.loc[:,'A1'],ahp_weights['A1'])
indicators_weighted[('Access','WQ')]=normalized_ahp.loc[:,('A3','WQ')]
indicators_weighted[('Access','DC')]=normalized_ahp.loc[:,('A3','DC')]
indicators_weighted[('Access','SS')]=weighted_geomean(normalized_ahp.loc[:,'A2'],ahp_weights['A2'])
indicators_weighted[('Economy','Eagr')]=weighted_geomean(normalized_ahp.loc[:,'E1'],ahp_weights['E1'])
indicators_weighted[('Economy','IE')]=normalized_ahp.loc[:,('E3','IE')]
indicators_weighted[('Economy','NRW')]=weighted_geomean(normalized_ahp.loc[:,'E2'],ahp_weights['E2'])

#  calculating dimensions & WSI
Dimensions_weightedgeo=pd.DataFrame(index=sub_indicators.index)
Dimensions_weightedgeo['Resource']=weighted_geomean(indicators_weighted[('Resource')],ahp_weights['R7'])
Dimensions_weightedgeo['Access']=weighted_geomean(indicators_weighted[('Access')],ahp_weights['A3'])
Dimensions_weightedgeo['Economy']=weighted_geomean(indicators_weighted[('Economy')],ahp_weights['E3'])
Dimensions_weightedgeo['WSI']=weighted_geomean(Dimensions_weightedgeo,ahp_weights['WSI']).round(decimals=4)
Dimensions_weightedgeo.sort_values('WSI',ascending=False)

,Resource,Access,Economy,WSI
province,,,,
mazan,0.794204,0.359494,0.616586,0.5984
gilan,0.927281,0.528230,0.246514,0.5964
lores,0.904152,0.332656,0.480961,0.5926
kord,0.827680,0.568352,0.274369,0.5886
chba,0.837131,0.604938,0.240812,0.5865
kohbu,0.774409,0.465845,0.332185,0.5595
ilam,0.783427,0.551135,0.232158,0.5480
kersh,0.713931,0.362570,0.287458,0.4846
arda,0.307823,0.486136,0.405742,0.3724


In [17]:
## Assessing water security using a mix of weighted_sum and weighted_geomean rules
normalized_lvl2=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
normalized_lvl2.columns=pd.MultiIndex.from_arrays([['R5'],['CV']], names=('Aggregation Level', 'Indicator'))
normalized_lvl2[('R5','CV')]=weighted_addgeo(normalized_ahp.loc[:,'R1'],ahp_weights['R1']).round(decimals=4)
normalized_lvl2[('R6','GD')]=weighted_addgeo(normalized_ahp.loc[:,'R2'],ahp_weights['R2']).round(decimals=4)
normalized_lvl2[('R6','GWD')]=weighted_addgeo(normalized_ahp.loc[:,'R3'],ahp_weights['R3']).round(decimals=4)

indicators_weighted=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
indicators_weighted.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_weighted[('Resource','IRWR')]=normalized_ahp.loc[:,('R7','IRWR')]
indicators_weighted[('Resource','CC')]=weighted_addgeo(normalized_lvl2['R5'].merge(normalized_ahp['R5'], left_index=True, right_index=True,how='outer'),ahp_weights['R5'])
indicators_weighted[('Resource','WS')]=weighted_addgeo(normalized_ahp.loc[:,'R4'],ahp_weights['R4'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]<0.8,('Resource','GW')]=weighted_addgeo(normalized_lvl2.loc[:,'R6'],ahp_weights['R6'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]>0.8,('Resource','GW')]=normalized_lvl2.loc[:,('R6','GD')]
indicators_weighted[('Access','DW')]=weighted_addgeo(normalized_ahp.loc[:,'A1'],ahp_weights['A1'])
indicators_weighted[('Access','WQ')]=normalized_ahp.loc[:,('A3','WQ')]
indicators_weighted[('Access','DC')]=normalized_ahp.loc[:,('A3','DC')]
indicators_weighted[('Access','SS')]=weighted_addgeo(normalized_ahp.loc[:,'A2'],ahp_weights['A2'])
indicators_weighted[('Economy','Eagr')]=weighted_addgeo(normalized_ahp.loc[:,'E1'],ahp_weights['E1'])
indicators_weighted[('Economy','IE')]=normalized_ahp.loc[:,('E3','IE')]
indicators_weighted[('Economy','NRW')]=weighted_addgeo(normalized_ahp.loc[:,'E2'],ahp_weights['E2'])

#  calculating dimensions & WSI
Dimensions_addgeo=pd.DataFrame(index=sub_indicators.index)
Dimensions_addgeo['Resource']=weighted_addgeo(indicators_weighted[('Resource')],ahp_weights['R7'])
Dimensions_addgeo['Access']=weighted_addgeo(indicators_weighted[('Access')],ahp_weights['A3'])
Dimensions_addgeo['Economy']=weighted_addgeo(indicators_weighted[('Economy')],ahp_weights['E3'])
Dimensions_addgeo['WSI']=weighted_addgeo(Dimensions_addgeo,ahp_weights['WSI']).round(decimals=4)
Dimensions_addgeo.sort_values('WSI',ascending=False)

,Resource,Access,Economy,WSI
province,,,,
lores,0.910298,0.473641,0.531917,0.6885
mazan,0.811367,0.459121,0.664456,0.6690
gilan,0.933205,0.575214,0.303492,0.6689
kord,0.839461,0.599195,0.399211,0.6643
chba,0.850782,0.644235,0.313657,0.6573
kohbu,0.833355,0.520961,0.409350,0.6400
ilam,0.812406,0.609895,0.312494,0.6302
kersh,0.743764,0.433889,0.351890,0.5576
azgh,0.448883,0.653317,0.346578,0.4804


In [18]:
# writer = pd.ExcelWriter('Dimensions & WSI.xlsx', engine='xlsxwriter')
# Dimensions_addminagg.to_excel(writer,sheet_name='Mean-Min')
# Dimensions_weightedsum.to_excel(writer,sheet_name='Mean')
# Dimensions_weightedgeo.to_excel(writer,sheet_name='Geo')
# Dimensions_addgeo.to_excel(writer,sheet_name='Mean-Geo')
# writer.save()